In [1]:
import cobra
import optlang
import optlang_enumerator.mcs_computation as mcs_computation
import numpy

In [2]:
ecc2 = cobra.io.read_sbml_model("ECC2comp.sbml")
# allow all reactions that are not boundary reactions as cuts (same as exclude_boundary_reactions_as_cuts option of compute_mcs)
cuts = numpy.array([not r.boundary for r in ecc2.reactions])
reac_id = ecc2.reactions.list_attr('id') # list of reaction IDs in the model
# define target (multiple targets are possible; each target can have multiple linear inequality constraints)
ecc2_mue_target = [[("Growth", ">=", 0.01)]] # one target with one constraint, a.k.a. syntehtic lethals
# this constraint alone would not be sufficient, but there are uptake limits defined in the reaction bounds
# of the model that are integerated below, first, however, parse the constraint(s) and convert to matrix format
ecc2_mue_target = [mcs_computation.relations2leq_matrix(
                   mcs_computation.parse_relations(t, reac_id_symbols=mcs_computation.get_reac_id_symbols(reac_id)), reac_id)
                   for t in ecc2_mue_target]
# now integrate the non-default reaction bounds from the network
mcs_computation.integrate_model_bounds(ecc2, ecc2_mue_target)
# this is just to show what the first (and only) target now looks like:
for c in mcs_computation.get_leq_constraints(ecc2, ecc2_mue_target)[0]:
    print(c)

da1f8112-c313-11eb-9b56-7824af3cf13a: -1.0*Growth + 1.0*Growth_reverse_699ae <= -0.01
da1f8113-c313-11eb-ba60-7824af3cf13a: 1.0*AcUp - 1.0*AcUp_reverse_36302 <= 10.0
da1f8114-c313-11eb-a757-7824af3cf13a: 1.0*CO2Up - 1.0*CO2Up_reverse_9ac6c <= 10.0
da1f8115-c313-11eb-8eae-7824af3cf13a: 1.0*GlcUp - 1.0*GlcUp_reverse_f2a50 <= 10.0
da1f8116-c313-11eb-88c1-7824af3cf13a: 1.0*GlycUp - 1.0*GlycUp_reverse_d1488 <= 10.0
da1f8117-c313-11eb-a475-7824af3cf13a: 1.0*SuccUp - 1.0*SuccUp_reverse_79869 <= 10.0


In [3]:
# calculate MCS up to size 3 with "any MCS" enumeration method
ecc2_mcs,_ = mcs_computation.compute_mcs(ecc2, ecc2_mue_target, cuts=cuts, enum_method=3, max_mcs_size=3, network_compression=True,                                              include_model_bounds=False) # include_model_bounds is disabled because it already has been done above
print(len(ecc2_mcs), "MCS found.")
# show MCS as n-tuples of reaction IDs
ecc2_mcs_rxns= [tuple(reac_id[r] for r in mcs) for mcs in ecc2_mcs]
print(ecc2_mcs_rxns)
# check that all MCS disable the first (and only) target
print(all(mcs_computation.check_mcs(ecc2, ecc2_mue_target[0], ecc2_mcs, optlang.interface.INFEASIBLE)))

Running FVA to find blocked reactions...
0.26599999999962165
Flipped EX_ca2_ex
Flipped EX_cl_ex
Flipped EX_cobalt2_ex
Flipped EX_cu2_ex
Flipped EX_fe2_ex
Flipped EX_fe3_ex
Flipped EX_glc_DASH_D_ex
Flipped EX_glyc_ex
Flipped EX_k_ex
Flipped EX_mg2_ex
Flipped EX_mn2_ex
Flipped EX_mobd_ex
Flipped EX_nh4_ex
Flipped EX_ni2_ex
Flipped EX_o2_ex
Flipped EX_pi_ex
Flipped EX_so4_ex
Flipped EX_zn2_ex
Flipped RPI
Removing 5 reactions:
[EX_q8_c, EX_adp_c, EX_nad_c, EX_mqn8_c, EX_nadp_c]
C:\Users\axel_2\AppData\Roaming\Python\Python37\site-packages\scipy\sparse\sputils.py:114 UserWarning: object dtype is not supported by sparse matrices
Running FVA for desired regions...
Using indicators.
Objective function is empty; set objective to self.minimize_sum_over_z
1.0
1.0
1.0
1.0
1.0
3.0
CS (11, 20, 28)
MCS (11, 20)
3.0
CS (13, 14, 20)
MCS (13, 14)
2.0
CS (30, 36)
MCS (30, 36)
2.0
CS (32, 66)
MCS (32, 66)
2.0
CS (6, 30)
MCS (6, 30)
3.0
CS (16, 21, 52)
MCS (21, 52)
2.0
CS (64, 66)
MCS (64, 66)
2.0
CS (56, 

In [4]:
# %% same calculation without network compression
ecc2_mcsF,_ = mcs_computation.compute_mcs(ecc2, ecc2_mue_target, cuts=cuts, enum_method=3, max_mcs_size=3, network_compression=False,
                                        include_model_bounds=False)
print(set(ecc2_mcs) == set(ecc2_mcsF)) # check that results agree

Running FVA to find blocked reactions...
0.125
Found 5 blocked reactions:
 ['EX_adp_c', 'EX_mqn8_c', 'EX_nad_c', 'EX_nadp_c', 'EX_q8_c']
Running FVA for desired regions...
Using indicators.
Objective function is empty; set objective to self.minimize_sum_over_z
1.0
1.0
1.0
1.0
1.0
1.0
1.0
2.0
CS (42, 43)
MCS (42, 43)
2.0
CS (67, 76)
MCS (67, 76)
3.0
CS (48, 64, 116)
MCS (48, 116)
3.0
CS (43, 51, 55)
MCS (51, 55)
2.0
CS (67, 75)
MCS (67, 75)
3.0
CS (51, 65, 95)
MCS (51, 95)
2.0
CS (69, 120)
MCS (69, 120)
3.0
CS (65, 101, 120)
MCS (101, 120)
3.0
CS (51, 66, 93)
MCS (66, 93)
3.0
CS (65, 88, 104)
MCS (88, 104)
2.0
CS (109, 117)
MCS (109, 117)
3.0
CS (80, 117, 120)
MCS (117, 120)
3.0
CS (65, 78, 120)
MCS (78, 120)
3.0
CS (80, 119, 120)
MCS (119, 120)
3.0
CS (63, 69, 109)
MCS (69, 109)
3.0
CS (78, 94, 117)
MCS (78, 117)
2.0
CS (109, 119)
MCS (109, 119)
2.0
CS (78, 109)
MCS (78, 109)
2.0
CS (58, 95)
MCS (58, 95)
2.0
CS (95, 102)
MCS (95, 102)
3.0
CS (81, 98, 114)
MCS (81, 98, 114)
3.0
CS (69, 